<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/05_crewAI/04_tasks_event_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automate Event Planning

In this notebook, we will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Crew, Task

In [ ]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [ ]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [ ]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks

- By using `output_json`, we can specify the structure of the output we want.
- By using `output_file`, we can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether we like the results or not) before finalising it.

In [ ]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json", # outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [ ]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [ ]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md", # outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since we set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [ ]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input.

**Note 2**:
- Since we set `human_input=True` for some tasks, the execution will ask for our input before it finishes running.
- When it asks for feedback, we use our mouse pointer to first click in the text box before typing anything.

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to carefully research and find the perfect venue in San Francisco that meets all the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: 36 Best Event Venues for Rent in San Francisco, CA - Peerspace
Link: https://www.peerspace.com/venues/san-francisco--ca/event-venue
Snippet: Choose from a wide range of event venues like elegant ballrooms, rustic workshops, picturesque gardens, and more. Not to mention, with the city's world-class ...
---
Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: The Presidio Chapel is a stunning and inspiring place for special events, with a breathtaking view

 [DEBUG]: == [Venue Coordinator] Task output: {
  "name": "The Conservatory at One Sansome",
  "address": "Downtown San Francisco",
  "capacity": 2046,
  "booking_status": "Available"
}


 [DEBUG]: == Working Agent: Logistics Manager
 [INFO]: == Starting Task: Coordinate catering and equipment for an event with 500 participants on 2024-09-15.
 [DEBUG]: == [Logistics Manager] Task output: {
  "name": "The Conservatory at One Sansome",
  "address": "Downtown San Francisco",
  "capacity": 2046,
  "booking_status": "Available"
}


 [DEBUG]: == Working Agent: Marketing and Communications Agent
 [INFO]: == Starting Task: Promote the Tech Innovation Conference aiming to engage at least500 potential attendees.
 [DEBUG]: == [Marketing and Communications Agent] Task output: {
  "name": "The Conservatory at One Sansome",
  "address": "Downtown San Francisco",
  "capacity": 2046,
  "booking_status": "Available"
}




> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chai

Thought: I need to gather information about catering and equipment setup for the event at The Conservatory at One Sansome.

Action: Read website content
Action Input: {"website_url": "https://www.conservatorysf.com/"} 

San Francisco's Premier Event Venue
Skip to main content
Spaces
Events sub-menu
					Events
Meetings & Conferences
Weddings & Celebrations
Nonprofit Parties & Fundraisers
Gallery
FAQ
Location
Our Team
Contact Us
Holbrook House
Toggle Navigation
Spaces
Events sub-menu
					Events
Meetings & Conferences
Weddings & Celebrations
Nonprofit Parties & Fundraisers
Gallery
FAQ
Location
Our Team
Contact Us
Holbrook House
Video Credit: IQ Videography
Careers
One Sansome Street, San Francisco, CA 94104
Instagram
LinkedIn
Email
powered by BentoBox
Home
Main content starts here, tab to start navigating
Modern — Classic.
hero video paused, press to play video
Playing hero video, press to pause video
Welcome to the Conservatory at One Sansome, a breathtaking new event space in the hear

This is the agent final answer: The Conservatory at One Sansome is a premier event venue located in downtown San Francisco. The venue offers a unique guest experience for weddings, corporate events, nonprofit galas, and fundraisers. The space consists of the Atrium, the Holbrook House bar and restaurant, and the North Lobby, totaling nearly 16,000 square feet of usable event space with a capacity of up to 2,046 guests.

The venue provides in-house catering specializing in American cuisine. Additionally, the venue has a preferred list of caterers available for selection. Equipment such as A/V equipment and air conditioning are also provided. The Conservatory is equipped to host a wide range of events and offers a one-of-a-kind experience in the heart of San Francisco.

For the event with 500 participants on 2024-09-15, the logistics for catering and equipment setup can be coordinated with the venue to ensure a seamless experience for all attendees.
Please provide a feedback: 

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Final Answer:
# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
- Researched strategies from various sources on boosting attendee engagement, such as leveraging interactive technology, personalizing experiences, fostering networking opportunities, gamifying the event, and providing engaging content.
- Read and gathered insights from the Event Tech Live website on boosting attendee engagement through innovative technology and personalized experiences.
- Explored various examples and case studies on how different events successfully engaged attendees through interactive elements.

## Attendee Engagement:
- Identified key strategies to boost attendee engagement, including utilizing interactive technology, personalizing experiences, fostering networking opportunities, gamifying the event, providing engaging content, leveraging social media, offering interactive workshops, gathering feedback, ensuring accessibility, and following up post-event.
- Recognized the

- Display the generated `venue_details.json` file.

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Downtown San Francisco',
 'booking_status': 'Available',
 'capacity': 2046,
 'name': 'The Conservatory at One Sansome'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If we try to run the code below before the file has been generated, our output would look like:

```
marketing_report.md
```

To display the `marketing_report.md`, wait some more and then try again.

In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md